<a href="https://colab.research.google.com/github/kailashnirola/python_notebooks/blob/main/perceptron_using_nn_module.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder

In [ ]:
df  = pd.read_csv('https://raw.githubusercontent.com/gscdit/Breast-Cancer-Detection/refs/heads/master/data.csv')
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [ ]:
df.drop(columns=['id', 'Unnamed: 32'],inplace=True)
df.shape

(569, 31)

In [ ]:
scaler = StandardScaler()
X = scaler.fit_transform(df.drop(columns=['diagnosis']))
encoder = LabelEncoder()
y = encoder.fit_transform(df['diagnosis'])
X = torch.tensor(X, dtype = torch.float32)
y = torch.tensor(y, dtype = torch.float32)

In [ ]:
X.shape, y.shape

(torch.Size([569, 30]), torch.Size([569]))

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
x_train.shape, x_test.shape, y_train.shape, y_test.shape

(torch.Size([455, 30]),
 torch.Size([114, 30]),
 torch.Size([455]),
 torch.Size([114]))

In [ ]:
class BreastCancerClassifier(nn.Module):
  def __init__(self, input_size):
    super().__init__()
    self.network = nn.Sequential(
      nn.Linear(input_size, 3),
      nn.ReLU(),
      nn.Linear(3, 1),
      nn.Sigmoid()
    )

  def forward(self, features):
    y_pred = self.network(features)
    return y_pred


In [ ]:
loss_func = nn.BCELoss()

In [ ]:
learning_rate = 0.2
epochs = 25

In [ ]:
y_pred.shape, y_train.shape

(torch.Size([455, 1]), torch.Size([455]))

In [ ]:
model = BreastCancerClassifier(X.shape[1])
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
for epoch in range(epochs):
  y_pred = model(x_train)
  loss = loss_func(y_pred, y_train.view(-1,1))
  print(f'Epochs: {epoch + 1}, Loss: {loss}')
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()

Epochs: 1, Loss: 0.618370771408081
Epochs: 2, Loss: 0.22754468023777008
Epochs: 3, Loss: 0.11788313090801239
Epochs: 4, Loss: 0.0927470251917839
Epochs: 5, Loss: 0.085325688123703
Epochs: 6, Loss: 0.0850219801068306
Epochs: 7, Loss: 0.08718730509281158
Epochs: 8, Loss: 0.08383509516716003
Epochs: 9, Loss: 0.07719626277685165
Epochs: 10, Loss: 0.07398904114961624
Epochs: 11, Loss: 0.07183321565389633
Epochs: 12, Loss: 0.06933904439210892
Epochs: 13, Loss: 0.0661950632929802
Epochs: 14, Loss: 0.06308615207672119
Epochs: 15, Loss: 0.059654947370290756
Epochs: 16, Loss: 0.0559878796339035
Epochs: 17, Loss: 0.052358001470565796
Epochs: 18, Loss: 0.049249317497015
Epochs: 19, Loss: 0.04733459651470184
Epochs: 20, Loss: 0.04609205946326256
Epochs: 21, Loss: 0.04563402757048607
Epochs: 22, Loss: 0.045874062925577164
Epochs: 23, Loss: 0.04627150669693947
Epochs: 24, Loss: 0.045674845576286316
Epochs: 25, Loss: 0.043756745755672455


#Model Evaluate

In [ ]:
with torch.no_grad():
  y_pred = model(x_test)
  y_pred = (y_pred > 0.5).float()
  acc = (y_pred == y_test).float().mean()
  print(f"Accuracy: {acc*100:.2f}")

Accuracy: 52.80


In [ ]:
!pip install torchinfo

In [ ]:
from torchinfo import summary
summary(model)

Layer (type:depth-idx)                   Param #
BreastCancerClassifier                   --
├─Sequential: 1-1                        --
│    └─Linear: 2-1                       93
│    └─ReLU: 2-2                         --
│    └─Linear: 2-3                       4
│    └─Sigmoid: 2-4                      --
Total params: 97
Trainable params: 97
Non-trainable params: 0